In [1]:
import findspark
findspark.init()
from typing import *
from pyspark.sql import *
from pyspark.sql.types import  IntegerType,ArrayType,StringType
from pyspark.ml import Pipeline
from pyspark.ml.feature import Word2Vec,Word2VecModel
from pyspark.sql.functions import *
import os

In [2]:
spark = SparkSession.builder.appName('enbbeding').master('local[*]').getOrCreate()

In [3]:
sent = ("a b " * 100 + "a c " * 10).split(" ")
doc = spark.createDataFrame([(sent,), (sent,)], ["sentence"])
doc

DataFrame[sentence: array<string>]

In [5]:
doc.show(10)

+--------------------+
|            sentence|
+--------------------+
|[a, b, a, b, a, b...|
|[a, b, a, b, a, b...|
+--------------------+



In [24]:
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="sentence")
model = word2Vec.fit(doc)

In [25]:
model.getVectors().show()

+----+--------------------+
|word|              vector|
+----+--------------------+
|   a|[0.09461779892444...|
|   b|[1.15474212169647...|
|   c|[-0.3794820010662...|
+----+--------------------+



In [26]:
doc.printSchema()

root
 |-- sentence: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [28]:
type(model) 

pyspark.ml.feature.Word2VecModel

In [49]:
df=model.getVectors()

In [57]:
# for i,j in df.
df.collect()[1]

Row(word='b', vector=DenseVector([1.1547, -0.5933, -0.8722, 0.4669, 0.5515]))

In [58]:
for row in df.collect():
    print(row['word'],row['vector'])

a [0.0946177989244461,-0.4951631426811218,0.06406556069850922,-0.37930983304977417,0.21593928337097168]
b [1.1547421216964722,-0.593326210975647,-0.8721810579299927,0.4669361710548401,0.551497220993042]
c [-0.3794820010662079,0.34077689051628113,0.06388652324676514,0.0352821946144104,-0.24136029183864594]


In [62]:
model.write().overwrite().save('./test_word2vec')

Py4JJavaError: An error occurred while calling o565.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:385)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFunctions$$